In [ ]:
import os
import sys

In [ ]:
import numpy as np
import scipy
import matplotlib.dates
import matplotlib.pyplot as plt
# from collections import defaultdict
import datetime
import dateutil
from dateutil import rrule

In [ ]:
package_root = os.path.abspath('../../..')
if package_root not in sys.path:
    sys.path.append(package_root)
from seismic.ASDFdatabase import FederatedASDFDataSet

In [ ]:
import obspy
from seismic.xcorqc.analytic_plot_utils import distance
from netCDF4 import Dataset as NCDataset

In [ ]:
# Imports for plotting
from textwrap import wrap
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
from seismic.xcorqc.xcorr_station_clock_analysis import plot_xcorr_file_clock_analysis

In [ ]:
ds = FederatedASDFDataSet.FederatedASDFDataSet("/g/data/ha3/Passive/SHARED_DATA/Index/asdf_files.txt")

In [ ]:
TIME_WINDOW = 300 # +/-
# TIME_WINDOW = 120 # +/-
# SNR_THRESHOLD = 10
SNR_THRESHOLD = 6

In [ ]:
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/ARMA_CMSA/AU.ARMA.AU.CMSA.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/KAKA_MTN/AU.KAKA.AU.MTN.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/MEEK_MORW/AU.MEEK.AU.MORW.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/MOO_TOO/AU.MOO.AU.TOO.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/MUN_NWAO/AU.MUN.AU.NWAO.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RKGY_NWAO/AU.RKGY.AU.NWAO.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/HTT_STKA/AU.HTT.AU.STKA.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/AU.RMQ.AU.EIDS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/YNG_CNB/AU.YNG.AU.CNB.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/WC1_WC2/AU.WC1.AU.WC2.nc"]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/YNG_CNB/AU.YNG.AU.CNB.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/WC1_WC2/AU.WC1.AU.WC2.nc"]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/KAKA_MTN/AU.KAKA.AU.MTN.nc"]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/MUN_NWAO/AU.MUN.AU.NWAO.nc"]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/KAKA_MTN/AU.KAKA.AU.MTN.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/MUN_NWAO/AU.MUN.AU.NWAO.nc"]

# Low band experiment:
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/KAKA_MTN/0.3-1.0/AU.KAKA.AU.MTN.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/MUN_NWAO/0.3-1.0/AU.MUN.AU.NWAO.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/YNG_CNB/0.3-1.0/AU.YNG.AU.CNB.nc"]

# Checking AU.ARMA [BHZ] against trusted AU.RMQ [SHZ]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/ARMA_RMQ/2012/0.3-1.0/AU.ARMA.AU.RMQ.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_RMQ/2013/0.3-1.0/AU.ARMA.AU.RMQ.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_RMQ/2014/0.3-1.0/AU.ARMA.AU.RMQ.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_RMQ/2015-16/0.3-1.0/AU.ARMA.AU.RMQ.nc"
#             ]

# Checking RMQ against EIDS
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2010-2011/AU.RMQ.AU.EIDS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2010-2011/AU.RMQ.AU.EIDS.2010-2011.nc"]
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2012/AU.RMQ.AU.EIDS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2013/AU.RMQ.AU.EIDS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2014/AU.RMQ.AU.EIDS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/RMQ_EIDS/2015/AU.RMQ.AU.EIDS.nc"
#             ]

# Checking ARMA against nearby AUPHS
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/AU/ARMA_AUPHS/2014/AU.ARMA.S.AUPHS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_AUPHS/2015/AU.ARMA.S.AUPHS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_AUPHS/2016/AU.ARMA.S.AUPHS.nc",
#              "/g/data/ha3/am7399/shared/xcorr/AU/ARMA_AUPHS/2017/AU.ARMA.S.AUPHS.nc"
#             ]

# Checking 7X.* against QIS
SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/7X/MA43_QIS/7X.MA43.AU.QIS.1.0-10.0.nc"]
tag = '(1.0-10.0 Hz)'

# Checking 7X.* against MTSU
# SRC_FILES = ["/g/data/ha3/am7399/shared/xcorr/7X/MB34_MTSU/7X.MB34.AU.MTSU.0.3-1.0.nc"]
# tag = '(0.3-1.0 Hz)'

save_plots = False
underlay_rcf_xcorr = False
for src_file in SRC_FILES:
    if save_plots:
        basename, _ = os.path.splitext(src_file)
        png_file = basename + ".png"
        plot_xcorr_file_clock_analysis(src_file, ds, TIME_WINDOW, SNR_THRESHOLD, png_file=png_file, show=False,
                                       underlay_rcf_xcorr=underlay_rcf_xcorr, title_tag=tag)
    else:
        plot_xcorr_file_clock_analysis(src_file, ds, TIME_WINDOW, SNR_THRESHOLD, 
                                       underlay_rcf_xcorr=underlay_rcf_xcorr, title_tag=tag)